In [3]:
import os
path = os.getcwd()
path = path.split('interactions', 1)[0] + 'interactions'
print(path)

/home/julia/Desktop/uni/enhancer-promoter-interactions


In [4]:
import pandas as pd

bed = pd.read_csv(path+'/results/whole_data/out_params_all_chrom.bedpe', delimiter='\t', header=None)
bed.columns = ['chr1', 'start1', 'end1', 'chr2', 'start2', 'end2', 'x', 'score', 'y', 'z', 'color']
print(bed.shape)
print(bed['chr1'].unique())
bed.head(10)


(1061804, 11)
['2L' '2R' '3L' '3R' '4' 'X']


,chr1,start1,end1,chr2,start2,end2,x,score,y,z,color
0,2L,10000910,10005105,2L,9950541,9951114,.,-0.004696,.,.,"255,0,0"
1,2L,10000910,10005105,2L,9951638,9952096,.,-0.005926,.,.,"255,0,0"
2,2L,10000910,10005105,2L,9953571,9954853,.,0.535730,.,.,"0,0,255"
3,2L,10000910,10005105,2L,9956462,9957908,.,0.569295,.,.,"0,0,255"
4,2L,10000910,10005105,2L,9963306,9963951,.,0.365345,.,.,"0,0,255"
5,2L,10000910,10005105,2L,9966422,9970768,.,0.552127,.,.,"0,0,255"
6,2L,10000910,10005105,2L,9972310,9973721,.,0.602123,.,.,"0,0,255"
7,2L,10000910,10005105,2L,9980748,9982050,.,0.022592,.,.,"0,0,255"
8,2L,10000910,10005105,2L,9982646,9985237,.,0.244502,.,.,"0,0,255"
9,2L,10000910,10005105,2L,9985950,9987615,.,-0.494903,.,.,"255,0,0"


In [5]:
bed['x'].replace('.', 0, inplace=True)
bed['y'].replace('.', 0, inplace=True)
bed['z'].replace('.', 0, inplace=True)
print(sum(bed['x']), sum(bed['y']), sum(bed['z']))
bed.drop(['x', 'y', 'z'], axis=1, inplace=True)
cicero_conns = bed.copy()
bed.head()

0 0 0


,chr1,start1,end1,chr2,start2,end2,score,color
0,2L,10000910,10005105,2L,9950541,9951114,-0.004696,"255,0,0"
1,2L,10000910,10005105,2L,9951638,9952096,-0.005926,"255,0,0"
2,2L,10000910,10005105,2L,9953571,9954853,0.535730,"0,0,255"
3,2L,10000910,10005105,2L,9956462,9957908,0.569295,"0,0,255"
4,2L,10000910,10005105,2L,9963306,9963951,0.365345,"0,0,255"


In [6]:
sum(cicero_conns['chr1'] != cicero_conns['chr2'])
cicero_conns = cicero_conns.drop('chr2', axis=1)
cicero_conns = cicero_conns.rename(columns={'chr1': 'chr'})
cicero_conns.head()

,chr,start1,end1,start2,end2,score,color
0,2L,10000910,10005105,9950541,9951114,-0.004696,"255,0,0"
1,2L,10000910,10005105,9951638,9952096,-0.005926,"255,0,0"
2,2L,10000910,10005105,9953571,9954853,0.535730,"0,0,255"
3,2L,10000910,10005105,9956462,9957908,0.569295,"0,0,255"
4,2L,10000910,10005105,9963306,9963951,0.365345,"0,0,255"


In [7]:
chromatin_loops = pd.read_excel(path+'/data/muszka/TableS2_Dmel_loops.xlsx')
chromatin_loops.head()

,loop ID,anchor ID,anchor chr,anchor start,anchor end,anchor summit,10-12h embryo neuron DHS count,loop size (in bp),anchor type,nearest TSS,anchor distance to TSS (in bp),meta-domain,loop clustering,loop type
0,L1,A1,chr2L,395670,395790,395730,1,16865113,intergenic,CG4213,6263,MD(L1-L3),clustered,meta-loop
1,L1,A19,chr2L,17260794,17260893,17260843,1,16865113,promoter,beat-IIIc,60,MD(L1-L3),clustered,meta-loop
2,L2,A1,chr2L,395670,395790,395730,1,16932336,intergenic,CG4213,6263,MD(L1-L3),clustered,meta-loop
3,L2,A20,chr2L,17328015,17328118,17328066,1,16932336,intergenic,CG45691,7303,MD(L1-L3),clustered,meta-loop
4,L3,A19,chr2L,17260794,17260893,17260843,1,67223,promoter,beat-IIIc,60,MD(L1-L3),clustered,intra-TAD loop


In [8]:
unique_col = ['loop ID', 'loop size (in bp)', 'loop clustering', 'loop type']
columns_to_duplicate = [col for col in list(chromatin_loops.columns) if col not in unique_col+['loop ID']]
print(columns_to_duplicate)
col_dict = {}
for col in columns_to_duplicate:
    col_dict[col] = ['first', 'last']
for col in unique_col:
    col_dict[col] = 'first'

['anchor ID', 'anchor chr', 'anchor start', 'anchor end', 'anchor summit', '10-12h embryo neuron DHS count', 'anchor type', 'nearest TSS', 'anchor distance to TSS (in bp)', 'meta-domain']


In [9]:
# join pairs of rows in chromatin_loops  with the same loop ID

print(chromatin_loops.shape)
chromatin_loops = chromatin_loops.groupby('loop ID').agg(col_dict)
chromatin_loops.head()


(136, 14)


anchor ID      anchor chr        anchor start           anchor end  \
            first last      first   last        first      last      first   
loop ID                                                                      
L1             A1  A19      chr2L  chr2L       395670  17260794     395790   
L10           A10  A13      chr2L  chr2L      6356962   9464540    6357076   
L11           A11  A12      chr2L  chr2L      6411271   9083037    6411420   
L12           A11  A14      chr2L  chr2L      6411271   9486317    6411420   
L13           A12  A14      chr2L  chr2L      9083037   9486317    9083252   

                  anchor summit            ...  nearest TSS             \
             last         first      last  ...        first       last   
loop ID                                    ...                           
L1       17260893        395730  17260843  ...       CG4213  beat-IIIc   
L10       9464794       6357019   9464720  ...       CG9500       numb   
L11       9083252       6411319   9083116  ...  DIP-epsilon     Toll-4   
L12       9486504       6411319   9486401  ...  DIP-epsilon        Gdi   
L13       9486504       9083116   9486401  ...       Toll-4        Gdi   

        anchor distance to TSS (in bp)         meta-domain               \
                                 first   last        first         last   
loop ID                                                                   
L1                                6263     60    MD(L1-L3)    MD(L1-L3)   
L10                               1241  14710  MD(L10,L12)  MD(L10,L12)   
L11                                  0    854      MD(L11)      MD(L11)   
L12                                  0   8291  MD(L10,L12)  MD(L10,L12)   
L13                                854   8291      MD(L13)      MD(L13)   

        loop ID loop size (in bp) loop clustering  loop type  
          first             first           first      first  
loop ID                                                       
L1           L1          16865113       clustered  meta-loop  
L10         L10           3107701       clustered  meta-loop  
L11         L11           2671797          single  meta-loop  
L12         L12           3075082       clustered  meta-loop  
L13         L13            403285          single  meta-loop  

[5 rows x 24 columns]

In [10]:
chromatin_loops.columns = chromatin_loops.columns.map('_'.join)
chromatin_loops.head()

,anchor ID_first,anchor ID_last,anchor chr_first,anchor chr_last,anchor start_first,anchor start_last,anchor end_first,anchor end_last,anchor summit_first,anchor summit_last,...,nearest TSS_first,nearest TSS_last,anchor distance to TSS (in bp)_first,anchor distance to TSS (in bp)_last,meta-domain_first,meta-domain_last,loop ID_first,loop size (in bp)_first,loop clustering_first,loop type_first
loop ID,,,,,,,,,,,,,,,,,,,,,
L1,A1,A19,chr2L,chr2L,395670,17260794,395790,17260893,395730,17260843,...,CG4213,beat-IIIc,6263,60,MD(L1-L3),MD(L1-L3),L1,16865113,clustered,meta-loop
L10,A10,A13,chr2L,chr2L,6356962,9464540,6357076,9464794,6357019,9464720,...,CG9500,numb,1241,14710,"MD(L10,L12)","MD(L10,L12)",L10,3107701,clustered,meta-loop
L11,A11,A12,chr2L,chr2L,6411271,9083037,6411420,9083252,6411319,9083116,...,DIP-epsilon,Toll-4,0,854,MD(L11),MD(L11),L11,2671797,single,meta-loop
L12,A11,A14,chr2L,chr2L,6411271,9486317,6411420,9486504,6411319,9486401,...,DIP-epsilon,Gdi,0,8291,"MD(L10,L12)","MD(L10,L12)",L12,3075082,clustered,meta-loop
L13,A12,A14,chr2L,chr2L,9083037,9486317,9083252,9486504,9083116,9486401,...,Toll-4,Gdi,854,8291,MD(L13),MD(L13),L13,403285,single,meta-loop


In [11]:
chromatin_loops.drop(columns=['anchor chr_last'], inplace=True)
chromatin_loops.rename(columns={'anchor chr_first':'chr', 
                                'anchor start_first':'start1', 
                                'anchor start_last':'start2',
                                'anchor end_first':'end1',
                                'anchor end_last':'end2'}, inplace=True)
loop_conns = chromatin_loops[['chr', 'start1', 'end1', 'start2', 'end2']].copy()
loop_conns.head()

,chr,start1,end1,start2,end2
loop ID,,,,,
L1,chr2L,395670,395790,17260794,17260893
L10,chr2L,6356962,6357076,9464540,9464794
L11,chr2L,6411271,6411420,9083037,9083252
L12,chr2L,6411271,6411420,9486317,9486504
L13,chr2L,9083037,9083252,9486317,9486504


In [15]:
loops_bigger = pd.read_csv(path+'/data/muszka/long_and_short_range_loops_D_mel.tsv', sep='\t')
loops_bigger.rename(columns={'chr1':'chr', 
                                'x1':'start1', 
                                'y1':'start2',
                                'x2':'end1',
                                'y2':'end2'}, inplace=True)

In [16]:
for col in ['start1', 'start2']:
    loop_conns.loc[:,'range_'+ col] = loop_conns.loc[:,col].apply(lambda x: max(0, x - 1000))
    loops_bigger.loc[:,'range_'+ col] = loops_bigger.loc[:,col].apply(lambda x: max(0, x - 1000))

for col in ['end1', 'end2']:
    loop_conns.loc[:,'range_' + col] = loop_conns.loc[:,col] + 1000
    loops_bigger.loc[:,'range_' + col] = loops_bigger.loc[:,col] + 1000
loop_conns['chr'] = loop_conns['chr'].apply(lambda x: x[3:])

loop_conns.head()

,chr,start1,end1,start2,end2,range_start1,range_start2,range_end1,range_end2
loop ID,,,,,,,,,
L1,2L,395670,395790,17260794,17260893,394670,17259794,396790,17261893
L10,2L,6356962,6357076,9464540,9464794,6355962,9463540,6358076,9465794
L11,2L,6411271,6411420,9083037,9083252,6410271,9082037,6412420,9084252
L12,2L,6411271,6411420,9486317,9486504,6410271,9485317,6412420,9487504
L13,2L,9083037,9083252,9486317,9486504,9082037,9485317,9084252,9487504


In [17]:
#change type to int
cicero_conns[['start1', 'start2', 'end1', 'end2']] = cicero_conns[['start1', 'start2', 'end1', 'end2']].astype(int)
loop_conns[['start1', 'start2', 'end1', 'end2']] = loop_conns[['start1', 'start2', 'end1', 'end2']].astype(int)
loops_bigger[['start1', 'start2', 'end1', 'end2']] = loops_bigger[['start1', 'start2', 'end1', 'end2']].astype(int)

In [18]:
def intersect_intervals(interval1, interval2):
    #print('dane', interval1, interval2)
    inter_start = max(interval1[0], interval2[0])
    inter_end = min(interval1[1], interval2[1])
    #print('przeciecie', inter_start, inter_end)
    if inter_start <= inter_end:
        return [inter_start, inter_end]
    else:
        return False

In [20]:
results = {'chr': [], 'intersection1': [], 
                                'intersection2': [],
                                'cicero1': [], 'cicero2': [], 
                                'loop1': [], 'loop2': [], 
                                'cicero_index': [], 'loop_index': []}

for chrom in set(cicero_conns['chr']).intersection(set(loop_conns['chr'])):
    cicero = cicero_conns[cicero_conns['chr'] == chrom].copy()
    loops = loop_conns[loop_conns['chr'] == chrom].copy()
    print(chrom)
    cicero['interval1'] = [list(x) for x in zip(cicero['start1'], cicero['end1'])]
    cicero['interval2'] = [list(x) for x in zip(cicero['start2'], cicero['end2'])]

    for l in loops.index:
        loop = loops.loc[l]
        loop_1 = [loop['range_start1'], loop['range_end1']]
        loop_2 = [loop['range_start2'], loop['range_end2']]
        finds1 = cicero[cicero['interval1'].apply(lambda x: intersect_intervals(x, loop_1))!=False]
        finds2 = finds1[finds1['interval2'].apply(lambda x: intersect_intervals(x, loop_2))!=False]
        if finds2.shape[0] != 0: 
            for find in finds2.index:
                results['chr'].append(chrom)
                results['cicero1'].append(finds2.loc[find, 'interval1'])
                results['cicero2'].append(finds2.loc[find, 'interval2'])
                results['intersection1'].append(intersect_intervals(finds2.loc[find, 'interval1'], loop_1))
                results['intersection2'].append(intersect_intervals(finds2.loc[find, 'interval2'], loop_2))
                results['loop1'].append(loop_1)
                results['loop2'].append(loop_2)
                results['cicero_index'].append(find)
                results['loop_index'].append(l)
            print(finds2.shape)





    

2R
3R
(2, 10)
(1, 10)
2L
(1, 10)
(1, 10)
X
(1, 10)
(1, 10)
3L


In [22]:
results_bigger_set = {'chr': [], 'intersection1': [], 
                                'intersection2': [],
                                'cicero1': [], 'cicero2': [], 
                                'loop1': [], 'loop2': [], 
                                'cicero_index': [], 'loop_index': []}

for chrom in set(cicero_conns['chr']).intersection(set(loops_bigger['chr'])):
    cicero = cicero_conns[cicero_conns['chr'] == chrom].copy()
    loops = loops_bigger[loops_bigger['chr'] == chrom].copy()
    print(chrom)
    cicero['interval1'] = [list(x) for x in zip(cicero['start1'], cicero['end1'])]
    cicero['interval2'] = [list(x) for x in zip(cicero['start2'], cicero['end2'])]

    for l in loops.index:
        loop = loops.loc[l]
        loop_1 = [loop['range_start1'], loop['range_end1']]
        loop_2 = [loop['range_start2'], loop['range_end2']]
        finds1 = cicero[cicero['interval1'].apply(lambda x: intersect_intervals(x, loop_1))!=False]
        finds2 = finds1[finds1['interval2'].apply(lambda x: intersect_intervals(x, loop_2))!=False]
        if finds2.shape[0] != 0: 
            for find in finds2.index:
                results_bigger_set['chr'].append(chrom)
                results_bigger_set['cicero1'].append(finds2.loc[find, 'interval1'])
                results_bigger_set['cicero2'].append(finds2.loc[find, 'interval2'])
                results_bigger_set['intersection1'].append(intersect_intervals(finds2.loc[find, 'interval1'], loop_1))
                results_bigger_set['intersection2'].append(intersect_intervals(finds2.loc[find, 'interval2'], loop_2))
                results_bigger_set['loop1'].append(loop_1)
                results_bigger_set['loop2'].append(loop_2)
                results_bigger_set['cicero_index'].append(find)
                results_bigger_set['loop_index'].append(l)
            print(finds2.shape)





    

2R
(8, 10)
(2, 10)
(2, 10)
(12, 10)
(12, 10)
(14, 10)
(30, 10)
(11, 10)
(6, 10)
(12, 10)
(16, 10)
(15, 10)
(8, 10)
(18, 10)
(23, 10)
(16, 10)
(18, 10)
(4, 10)
(6, 10)
(30, 10)
(24, 10)
3R
(2, 10)
(1, 10)
(12, 10)
(6, 10)
(5, 10)
(6, 10)
(2, 10)
(15, 10)
(6, 10)
(2, 10)
(20, 10)
(10, 10)
(8, 10)
(2, 10)
(2, 10)
(18, 10)
(4, 10)
(6, 10)
(20, 10)
(16, 10)
(5, 10)
(12, 10)
(16, 10)
(10, 10)
(4, 10)
(25, 10)
(8, 10)
(8, 10)
(8, 10)
(7, 10)
(2, 10)
(9, 10)
(20, 10)
2L
(1, 10)
(1, 10)
(18, 10)
(20, 10)
(10, 10)
(6, 10)
(6, 10)
(6, 10)
(15, 10)
(10, 10)
(3, 10)
(21, 10)
(25, 10)
(8, 10)
(3, 10)
(9, 10)
(3, 10)
(8, 10)
(9, 10)
(6, 10)
(12, 10)
(9, 10)
(15, 10)
(10, 10)
(16, 10)
(11, 10)
(72, 10)
(4, 10)
X
(1, 10)
(1, 10)
(2, 10)
(1, 10)
(30, 10)
(20, 10)
(6, 10)
(1, 10)
(6, 10)
(6, 10)
(72, 10)
(8, 10)
(2, 10)
(18, 10)
(10, 10)
(29, 10)
(2, 10)
(9, 10)
(3, 10)
(3, 10)
(4, 10)
(4, 10)
3L
(25, 10)
(16, 10)
(1, 10)
(8, 10)
(4, 10)
(16, 10)
(12, 10)
(75, 10)
(15, 10)
(29, 10)
(15, 10)
(15, 10)
(28,

In [23]:
pd.DataFrame(results_bigger_set)

,chr,intersection1,intersection2,cicero1,cicero2,loop1,loop2,cicero_index,loop_index
0,2R,"[6570000, 6570193]","[6607004, 6608420]","[6568533, 6570193]","[6607004, 6608420]","[6570000, 6582000]","[6607000, 6617000]",364729,112
1,2R,"[6570000, 6570193]","[6609890, 6614372]","[6568533, 6570193]","[6609890, 6614372]","[6570000, 6582000]","[6607000, 6617000]",364730,112
2,2R,"[6570000, 6570193]","[6615136, 6615478]","[6568533, 6570193]","[6615136, 6615478]","[6570000, 6582000]","[6607000, 6617000]",364731,112
3,2R,"[6570000, 6570193]","[6616178, 6616673]","[6568533, 6570193]","[6616178, 6616673]","[6570000, 6582000]","[6607000, 6617000]",364732,112
4,2R,"[6570794, 6579620]","[6607004, 6608420]","[6570794, 6579620]","[6607004, 6608420]","[6570000, 6582000]","[6607000, 6617000]",364773,112
...,...,...,...,...,...,...,...,...,...
1713,3L,"[22000067, 22001075]","[22040502, 22041018]","[22000067, 22001075]","[22040502, 22041018]","[21987000, 22003000]","[22025000, 22042000]",525448,176
1714,3L,"[22001710, 22002862]","[22025000, 22025575]","[22001710, 22002862]","[22023724, 22025575]","[21987000, 22003000]","[22025000, 22042000]",525493,176
1715,3L,"[22001710, 22002862]","[22032561, 22032793]","[22001710, 22002862]","[22032561, 22032793]","[21987000, 22003000]","[22025000, 22042000]",525494,176
1716,3L,"[22001710, 22002862]","[22034010, 22035268]","[22001710, 22002862]","[22034010, 22035268]","[21987000, 22003000]","[22025000, 22042000]",525495,176


In [25]:
pd.DataFrame(results)

,chr,intersection1,intersection2,cicero1,cicero2,loop1,loop2,cicero_index,loop_index
0,2L,"[17260454, 17261147]","[17327015, 17328748]","[17260454, 17261147]","[17326683, 17328748]","[17259794, 17261893]","[17327015, 17329118]",78178,L3
1,2L,"[2611335, 2613163]","[2676575, 2678786]","[2611335, 2613163]","[2674974, 2679304]","[2611166, 2613266]","[2676575, 2678786]",131529,L7
2,X,"[945156, 946632]","[985754, 986993]","[945156, 946632]","[985754, 986993]","[944883, 946994]","[985055, 987686]",1056189,L64
3,X,"[985754, 986993]","[1009243, 1010923]","[985754, 986993]","[1009243, 1010923]","[985055, 987686]","[1008817, 1011125]",1060922,L66
4,3R,"[12020934, 12021424]","[12093666, 12095330]","[12020934, 12021424]","[12093307, 12095330]","[12020090, 12022281]","[12093666, 12095798]",631887,L51
5,3R,"[12021946, 12022281]","[12093666, 12095330]","[12021946, 12022480]","[12093307, 12095330]","[12020090, 12022281]","[12093666, 12095798]",631928,L51
6,3R,"[29343951, 29346515]","[29375598, 29377948]","[29343951, 29346515]","[29372042, 29377948]","[29342984, 29347065]","[29375598, 29378448]",790353,L60


In [26]:
cicero_conns[cicero_conns['intersection'] == 'y'].shape

(7, 8)

In [24]:
pd.DataFrame(results).to_csv(path+'/results/whole_data/loop_cicero_intersections.csv')
pd.DataFrame(results_bigger_set).to_csv(path+'/results/whole_data/intersection_results_bigger_set.csv')

In [25]:
loop_conns.to_csv(path+'/results/whole_data/TableS2_Dmel_loops_formatted.csv')
loops_bigger.to_csv(path+'/results/whole_data/long_and_short_range_loops_D_mel_formated.csv')